credit cart balance は SK_ID_CURR に対して missing value が多いので、
あまり効かないかもしれない。

In [6]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
FILEPATH_CREDIT_CARD_BALANCE   = "../data/credit_card_balance.csv.f"

In [8]:
df_credit_balance = pd.read_feather(FILEPATH_CREDIT_CARD_BALANCE)

In [9]:
df_credit_balance.head(5)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [27]:
def df_credit_balance_make_feature(df_org, duration, feat_suffix, filename):
    df_dur = df_org[df_org['MONTHS_BALANCE'] >=duration]
    
    df_credit_balance_ec = pd.get_dummies(df_dur)
    oh_cols = ['SK_ID_CURR']
    for col in df_credit_balance_ec.columns:
        if 'NAME_CONTRACT_STATUS' in col:
            oh_cols.append(col)
    
    df_credit_balance_ec = df_credit_balance_ec[oh_cols]
    df_credit_balance_ec_sum = df_credit_balance_ec.groupby('SK_ID_CURR', as_index=False).sum()
    df_credit_balance_ec_avg = df_credit_balance_ec.groupby('SK_ID_CURR', as_index=False).mean()

    ### rename columns
    df_credit_balance_ec_sum.columns = ['sum_' + f_ for f_ in df_credit_balance_ec_sum.columns]
    df_credit_balance_ec_sum.rename(columns={'sum_SK_ID_CURR': 'SK_ID_CURR'}, inplace=True)

    df_credit_balance_ec_avg.columns = ['avg_' + f_ for f_ in df_credit_balance_ec_avg.columns]
    df_credit_balance_ec_avg.rename(columns={'avg_SK_ID_CURR': 'SK_ID_CURR'}, inplace=True)

    ### count the number of previous applications for a given ID
    nb_prevs = df_dur[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
    df_dur['SK_ID_PREV'] = df_dur['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])

    ### average of all other columns 
    feat_cc_bal = df_dur.groupby('SK_ID_CURR', as_index=False).mean()

    ### merge categorical features
    feat_cc_bal = pd.merge(feat_cc_bal, df_credit_balance_ec_sum, on='SK_ID_CURR', how='left')
    feat_cc_bal = pd.merge(feat_cc_bal, df_credit_balance_ec_avg, on='SK_ID_CURR', how='left')

    ### rename columns
    feat_cc_bal.columns = ['cc_bal_' + feat_suffix +  f_ for f_ in feat_cc_bal.columns]
    for col in feat_cc_bal.columns:
        if 'SK_ID_CURR' in col:
            feat_cc_bal.rename(columns={col: 'SK_ID_CURR'}, inplace=True)
    
    ### save feature to file
    feat_cc_bal.to_feather('../data/' + filename + '.csv.f')
    

In [28]:
df_credit_balance_make_feature(df_credit_balance, -3, '_3m',  'feature_cc_balance_3m_0824_1')
df_credit_balance_make_feature(df_credit_balance, -3, '_6m',  'feature_cc_balance_6m_0824_1')
df_credit_balance_make_feature(df_credit_balance, -3, '_12m', 'feature_cc_balance_12m_0824_1')
